In [1]:
        !git clone https://github.com/AK18k/lora

Cloning into 'lora'...
remote: Enumerating objects: 1369, done.
remote: Counting objects: 100% (1369/1369), done.
remote: Compressing objects: 100% (975/975), done.
remote: Total 1369 (delta 365), reused 1358 (delta 359), pack-reused 0
Receiving objects: 100% (1369/1369), 23.86 MiB | 21.03 MiB/s, done.
Resolving deltas: 100% (365/365), done.


In [6]:
!pip install progress

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for progress: filename=progress-1.6-py3-none-any.whl size=9611 sha256=45621f354eb22cdf0cc34a3223fae441db596bf349b30b72992f4ed5526a554a
  Stored in directory: /root/.cache/pip/wheels/a2/68/5f/c339b20a41659d856c93ccdce6a33095493eb82c3964aac5a1
Successfully built progress


In [2]:
# !cd ./lora/examples/NLG
# !ls '*.sh'
!bash ./lora/examples/NLG/download_pretrained_checkpoints.sh

downloading pretrained model checkpoints...
--2023-06-03 09:17:48--  https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-pytorch_model.bin
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.103.158, 54.231.166.88, 52.217.161.144, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.103.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 548118077 (523M) [application/octet-stream]
Saving to: ‘gpt2-pytorch_model.bin’

gpt2-pytorch_model. 100%[===================>] 522.73M  51.6MB/s    in 9.9s    

2023-06-03 09:17:58 (52.6 MB/s) - ‘gpt2-pytorch_model.bin’ saved [548118077/548118077]

--2023-06-03 09:17:58--  https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-medium-pytorch_model.bin
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.106.14, 52.217.236.232, 52.217.83.238, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.106.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520013706 (1.4G) [appl

In [7]:
!bash ./lora/examples/NLG/create_datasets.sh

creating e2e datasets...
train...
test...
valid...
creating webnlg datasets...
train...
cate Airport 1090
cate Astronaut 530
cate Building 972
cate City 243
cate ComicsCharacter 285
cate Food 1424
cate Monument 267
cate SportsTeam 786
cate University 406
cate WrittenWork 937
test...
cate Airport 136
cate Monument 33
cate Building 120
cate WrittenWork 116
cate SportsTeam 98
cate University 51
cate Astronaut 66
cate ComicsCharacter 35
cate City 139
cate Food 177
cate Politician 209
cate MeanOfTransportation 198
cate Athlete 159
cate Artist 213
cate CelestialBody 112
valid...
cate Airport 136
cate Astronaut 67
cate Building 123
cate City 31
cate ComicsCharacter 37
cate Food 178
cate Monument 32
cate SportsTeam 99
cate University 51
cate WrittenWork 118
creating dart datasets...
train...
unique source is 30526
test...
unique source is 6959
valid...
unique source is 2768
script complete!


In [10]:
!bash ./lora/examples/NLG/eval/download_evalscript.sh


./lora/examples/NLG/eval/download_evalscript.sh: line 3: cd: eval: No such file or directory
installing evaluation dependencies
downloading e2e-metrics...
Cloning into 'e2e'...
remote: Enumerating objects: 909, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 909 (delta 0), reused 1 (delta 0), pack-reused 906
Receiving objects: 100% (909/909), 106.78 MiB | 33.98 MiB/s, done.
Resolving deltas: 100% (492/492), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
downloading GenerationEval for webnlg and dart...
Cloning into 'GenerationEval'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 82 (delta 36), reused 51 (delta 15), pack-reused 0
Unpacking objects: 100% (82/82), 221.17 KiB | 4.51 MiB/s, done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/cola